In [0]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
import requests
import json

headers = requests.utils.default_headers()
headers.update({
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
})

cars = [
    "ford",
    "bmw",
    "fiat",
    "toyota",
    "peugeot",
    "audi",
    "mercedes",
    "volvo",
    "nissan",
    "suzuki",
    "volkswagen",
    "renault",
    "skoda",
    "opel"
]

# POLAND

---



---





In [0]:
pages_pl = []

for i in cars:
  URL_PL = "https://www.otomoto.pl/osobowe/" + i + "/?search%5Bnew_used%5D=on"

  page_html = requests.get(URL_PL, headers=headers)
  page_html = soup(page_html.content, "lxml")
  pages_pl.append(page_html)

In [0]:
f = open("cars_PL.csv", "w")
csv_headers = "country;model;name;year;mileage;fuel;price\n"
f.write(csv_headers)

for page in pages_pl:
  info_divs = page.findAll("div", {"class":"ds-details-container"})
  # print(info)
  for info_div in info_divs:
    name = info_div.find("a", { "class":"offer-title__link"}).get("title")

    model = [j for j in cars if j in name.lower()]
    if "Škoda" in name: model = ["skoda"]
    if len(model) != 0: model = model[0]
    else: break

    year = info_div.find("li", { "data-code":"year"}).span.text
    mileage = info_div.find("li", { "data-code":"mileage"}).span.text[:-2]
    fuel = info_div.find("li", { "data-code":"fuel_type"}).span.text
    price = info_div.find("span", {"class":"ds-price-number"}).span.text
    
    f.write(f"PL;{model};{name};{year};{mileage};{fuel};{price}\n")

f.close()

# GREAT BRITAIN

---



---



In [0]:
pages_uk = []

for i in cars:
  URL_UK = "https://www.autotrader.co.uk/car-search?advertClassification=standard&make=" + i.upper() + "&postcode=BS11GB&onesearchad=Used&onesearchad=Nearly%20New&onesearchad=New&advertising-location=at_cars&is-quick-search=TRUE&page=1"

  page_html = requests.get(URL_UK, headers=headers)
  page_html = soup(page_html.content, "lxml")
  pages_uk.append(page_html)


In [0]:
f = open("cars_UK.csv", "w")
csv_headers = "country;model;name;year;mileage;fuel;price\n"
f.write(csv_headers)


for page in pages_uk:
  info_divs = page.findAll("article", {"class":"sso-listing"})
  for info_div in info_divs:
    name = info_div.find("h2", { "class":"listing-title"}).a.text.replace(';', "").replace(',', "")
    # print(name)

    model = [j for j in cars if j in name.lower()]
    if "Škoda" in name: model = ["skoda"]
    if len(model) != 0: model = model[0]
    else: break

    auto_specs = info_div.find("ul", { "class":"listing-key-specs"}).findAll("li")
    for y in auto_specs:
      if "20" in y.text:
        year = y.text[0:4].strip()
      if "miles" in y.text:
        mileage = y.text[:-5].replace(",", " ").strip()
      if "petrol" in y.text.lower() or "diesel" in y.text.lower():
        fuel = y.text
    price = info_div.find("div", {"class":"vehicle-price"}).text.replace("£", "").strip()
    price = price.replace(',', '')

    try:
      price = int(price)
    except:
      break

    f.write(f"UK;{model};{name};{year};{mileage};{fuel};{price}\n")

f.close()

# GERMANY

---
---



In [0]:
pages_de = []

for i in cars:
  URL_DE = "https://www.autoscout24.com/lst/"+ i +"?sort=age&desc=1&ustate=N%2CU&size=20&page=1&priceto=25000&pricefrom=500&atype=C&"

  page_html = requests.get(URL_DE, headers=headers)
  page_html = soup(page_html.content, "lxml")
  pages_de.append(page_html)


In [0]:
f = open("cars_DE.csv", "w")
csv_headers = "country;model;name;year;mileage;fuel;price\n"
f.write(csv_headers)


for page in pages_de:
  info_divs = page.findAll("div", {"class":"cldt-summary-full-item-main"})
  for info_div in info_divs:
    name = info_div.find("h2", { "class":"cldt-summary-makemodel"}).text
    model = [j for j in cars if j in name.lower()]
    if len(model) != 0: model = model[0]
    else: break

    auto_specs = info_div.find("ul", { "data-item-name":"vehicle-details"}).findAll("li")
    year = auto_specs[1].text[4:].strip()
    if year in "(First Registration)": break
    mileage = auto_specs[0].text[:-3].replace(",", "").strip()
    try:
      mileage = int(mileage)
    except:
      break

    fuel = auto_specs[6].text.strip()

    price = info_div.find("span", {"class":"cldt-price"}).text.replace("€", "").replace(".-", "").replace(",", " ").replace("VAT deductible", "").strip()

    f.write(f"DE;{model};{name};{year};{mileage};{fuel};{price}\n")

f.close()

# UNITED STATES

---
---


In [0]:
pages_us = []

for i in cars:
  URL_US ="https://www.auto.com/cars-for-sale/"+i+"?range=30&zc=90120"

  page_html = requests.get(URL_US, headers=headers)
  page_html = soup(page_html.content, "lxml")
  pages_us.append(page_html)


In [0]:
url_list_us = []

for page in pages_us:
  scripts = page.findAll("script", {"type":"application/ld+json"})[2:]
  for scr in scripts[:1]:
    json_data = json.loads(str(scr.text))
    json_data = json_data['itemListElement']
    for json_url in json_data:
      url_list_us.append(json_url['url'])

In [0]:
f = open("cars_US.csv", "w")
csv_headers = "country;model;name;year;mileage;price\n"
f.write(csv_headers)

for i in url_list_us:
  URL_US = str(i)

  page_html = requests.get(URL_US, headers=headers)
  page_html = soup(page_html.content, "lxml")
  for line in page_html.text.split('\n'):
    if "auto_stats_vhr_report_enabled" in line:
      d = line.find("DEAL")
      line = line[d+4:len(line)]
      try:
        line = json.loads(line)
      except:
        break

      mileage = line["mileageFromOdometer"].replace(',', "").replace("miles", "")
      model = line['brand']['name'].lower()
      name = line['name']
      price = line['offers']['price']
      year = line['modelDate']

      f.write(f"US;{model};{name};{year};{mileage};{price}\n")

f.close()

# THE NETHERLANDS

---



---



In [0]:
pages_nl = []

for i in cars:
  URL_NL = "https://www.trucksnl.com/passenger-cars/" + i
  for j in range(1, 6):
    URL = URL_NL + "?page=" + str(j)
    page_html = requests.get(URL, headers=headers)
    page_html = soup(page_html.content, "html")
    pages_nl.append(page_html)


In [0]:
f = open("cars_NL.csv", "w")
csv_headers = "country;model;name;year;mileage;fuel;price\n"
f.write(csv_headers)

for page in pages_nl:
  info_divs = page.findAll("div", {"class":"no-gutters"})
  for info_div in info_divs[15:]:
    name = info_div.find("span", { "class":"type"})
    name = name.text.strip()
    model = [j for j in cars if j in name.lower()]
    model = model[0]

    auto_specs = info_div.find("ul", { "class":"list-properties"}).findAll("li")

    year = auto_specs[0].text.strip()
    mileage = auto_specs[1].text[:-3].replace(",", "").strip()
    fuel = auto_specs[3].text.strip()

    try:
      year = int(year)
      mileage = int(mileage)
      price = info_div.find("strong", {"class":"price"}).text.strip().replace("€", "").replace(",", "")
    except:
      break
    

    f.write(f"NL;{model};{name};{year};{mileage};{fuel};{price}\n")

f.close()